In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 5,
    "exp_name": ct.generate_readable_short_id("Laplace"), #"main_run5_cv5_21data_100trees_40opt_fast",
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 5,
    "plot": False,
    "calib_methods": ["RF_d", "RF_opt",
                      "Platt", "ISO", "Beta", "CRF", "VA",
                      "tlr", "Rank", #"Elkan"
                      # "RF_ens_k", "RF_ens_r", 
                      "RF_large",
                      ],
    
    "metrics": ["acc", "brier", "ece", "logloss"],

    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": np.arange(2, 100).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf": np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 1, # if set to 1, OOB must be false

    "exp_key": "data_name",
    "exp_values": [
                  "vertebral", 
                  "wilt",
                  "parkinsons", 
                  "heart",
                  "wdbc",
                  "bank", 
                  "ionosphere", 
                  "HRCompetencyScores",
                  "spambase", 
                  "QSAR", 
                  "diabetes", 
                  "breast", 
                  "SPF",
                  "hillvalley",
                  "pc4",
                  "scene",
                  "Sonar_Mine_Rock_Data",
                  "Customer_Churn",
                  "jm1",
                  "eeg",
                  # "madelon",
                  # "phoneme",
                  # "nomao",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param vertebral done
exp_param wilt done
exp_param parkinsons done
exp_param heart done
exp_param wdbc done
exp_param bank done
exp_param ionosphere done
exp_param HRCompetencyScores done
exp_param spambase done
exp_param QSAR done
exp_param diabetes done
exp_param breast done
exp_param SPF done
exp_param hillvalley done
exp_param pc4 done
exp_param scene done
exp_param Sonar_Mine_Rock_Data done
exp_param Customer_Churn done


/Users/mhshaker/Documents/PhD/Projects/RF_ranking_calibration/RFcalib/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


exp_param jm1 done


/Users/mhshaker/Documents/PhD/Projects/RF_ranking_calibration/RFcalib/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


exp_param eeg done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [5]:
tables["brier"].round(4)


,RF_d,RF_opt,Platt,ISO,Beta,CRF,VA,tlr,Rank,RF_large
Data,,,,,,,,,,
vertebral,0.1148,0.1096,0.1223,0.1280,0.1226,0.1218,0.1215,0.1273,0.1459,0.1133
wilt,0.0147,0.0126,0.0137,0.0136,0.0133,0.0131,0.0133,0.0143,0.0173,0.0121
parkinsons,0.0849,0.0982,0.1030,0.1083,0.1072,0.1034,0.1024,0.1086,0.1295,0.0942
heart,0.1396,0.1401,0.1401,0.1501,0.1414,0.1415,0.1445,0.1440,0.1533,0.1338
wdbc,0.0366,0.0362,0.0387,0.0423,0.0391,0.0392,0.0408,0.0393,0.0559,0.0345
bank,0.0090,0.0090,0.0086,0.0104,0.0096,0.0093,0.0098,0.0098,0.0245,0.0079
ionosphere,0.0634,0.0656,0.0664,0.0737,0.0679,0.0668,0.0700,0.0685,0.0872,0.0626
HRCompetencyScores,0.0690,0.0704,0.0741,0.0782,0.0767,0.0727,0.0757,0.0774,0.0884,0.0695
spambase,0.0476,0.0486,0.0449,0.0455,0.0450,0.0467,0.0452,0.0496,0.0461,0.0453


In [6]:
ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}")

metric acc
Friedman Test Statistic: 110.4222222222223
P-value: 1.2063245240949563e-19
The differences between groups are significant.
metric brier
Friedman Test Statistic: 98.43272727272733
P-value: 3.263525304924237e-17
The differences between groups are significant.
metric ece
Friedman Test Statistic: 45.436363636363694
P-value: 7.660703543576191e-07
The differences between groups are significant.
metric logloss
Friedman Test Statistic: 102.51272727272737
P-value: 4.877482093279565e-18
The differences between groups are significant.
